Датасет: https://www.kaggle.com/datasets/rajyellow46/wine-quality

# Качество вина

Набор данных был загружен из репозитория машинного обучения UCI.

Два набора данных относятся к красному и белому вариантам португальского вина «Виньо Верде». Ссылка [Кортез и др., 2009]. Из соображений конфиденциальности и логистики доступны только физико-химические (входные) и сенсорные (выходные) переменные (например, отсутствуют данные о сортах винограда, марке вина, отпускной цене вина и т. д.).

Эти наборы данных можно рассматривать как задачи классификации или регрессии. Классы упорядочены и не сбалансированы (например, нормальных вин намного больше, чем отличных или плохих). Алгоритмы обнаружения выбросов можно использовать для обнаружения нескольких отличных или плохих вин. Кроме того, мы не уверены, все ли входные переменные актуальны. Поэтому было бы интересно протестировать методы выбора функций.

Два набора данных были объединены, и несколько значений были удалены случайным образом.

**Цель исследования:** восстановить удаленные данные.

**Ход исследования:**
1. Обзор данных
2. Предобработка данных
3. Проверка гипотез

## 1. Обзор данных

In [1]:
#импортируем библиотеки
import os
import pandas as pd

In [2]:
#загрузка файла с kaggle
def load_data_from_kaggle():
    data_path = 'wine-quality-data'

    if not os.path.exists(data_path):
        !kaggle datasets download -d rajyellow46/wine-quality        
        with zipfile.ZipFile('wine-quality.zip', 'r') as zipp:
            zipp.extractall(data_path)

    train_path = "\\".join([data_path, os.listdir(data_path)[0]])
    
    return pd.read_csv(train_path)

In [3]:
#выводим на экран первые 10 строк
df = load_data_from_kaggle()
df.head(10)

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
5,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
6,white,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,6
7,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
8,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
9,white,8.1,0.22,0.43,1.5,0.044,28.0,129.0,0.9938,3.22,0.45,11.0,6


In [4]:
#выводим общую информацию о таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   type                  6497 non-null   object 
 1   fixed acidity         6487 non-null   float64
 2   volatile acidity      6489 non-null   float64
 3   citric acid           6494 non-null   float64
 4   residual sugar        6495 non-null   float64
 5   chlorides             6495 non-null   float64
 6   free sulfur dioxide   6497 non-null   float64
 7   total sulfur dioxide  6497 non-null   float64
 8   density               6497 non-null   float64
 9   pH                    6488 non-null   float64
 10  sulphates             6493 non-null   float64
 11  alcohol               6497 non-null   float64
 12  quality               6497 non-null   int64  
dtypes: float64(11), int64(1), object(1)
memory usage: 660.0+ KB


В таблице 13 столбцов. Тип данных в столбцах - `object`, `float64`, `int64`. Количество значений в столбцах различается, что подтверждает наличие пропусков.

Согласно документации к данным:

* `type` — тип вина;
* `fixed acidity` — фиксированная кислотность;
* `volatile acidity` — летучая кислотность;
* `citric acid` — лимонная кислота;
* `residual sugar` — остаточный сахар;
* `chlorides` - хлориды;
* `free sulfur dioxide` — свободный диоксид серы;
* `total sulfur dioxide` — общий диоксид серы;
* `density` — плотность;
* `pH` — pH;
* `sulphates` — сульфаты;
* `alcohol` — спирт;
* `quality` — качество (оценка от 0 до 10);

**Вывод:**

Обнаружены пропуски данных в следующих столбцах: `fixed acidity`, `volatile acidity`, `citric acid`, `residual sugar`, `chlorides`, `pH`, `sulphates`, `sulphates`.

Также переименуем названия столбцов в стиль snake_case.

## 2. Предобработка данных

### 2.1 Стиль заголовков

In [5]:
df.rename(columns={elem: '_'.join(elem.split(' ')) for elem in list(df.columns)}, inplace=True)
df.head()

,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


### 2.2 Пропуски значений

In [6]:
#считаем кол-во прропусков
df.isna().sum()

type                     0
fixed_acidity           10
volatile_acidity         8
citric_acid              3
residual_sugar           2
chlorides                2
free_sulfur_dioxide      0
total_sulfur_dioxide     0
density                  0
pH                       9
sulphates                4
alcohol                  0
quality                  0
dtype: int64

Посмотрим на пропуски в столбце `fixed_acidity`, видим что пропуски содержатся по двум типам вина - красное и белое. Заменим пустые значения на среднее значение по типам вина.

In [7]:
df[df['fixed_acidity'].isna()]

,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
17,white,NaN,0.660,0.48,1.2,0.029,29.0,75.0,0.98920,3.33,0.39,12.8,8
174,white,NaN,0.270,0.31,17.7,0.051,33.0,173.0,0.99900,3.09,0.64,10.2,5
249,white,NaN,0.410,0.14,10.4,0.037,18.0,119.0,0.99600,3.38,0.45,10.0,5
267,white,NaN,0.580,0.07,6.9,0.043,34.0,149.0,0.99440,3.34,0.57,9.7,5
368,white,NaN,0.290,0.48,2.3,0.049,36.0,178.0,0.99310,3.17,0.64,10.6,6
518,white,NaN,0.130,0.28,1.9,0.050,20.0,78.0,0.99180,3.43,0.64,10.8,6
1079,white,NaN,NaN,0.29,6.2,0.046,29.0,227.0,0.99520,3.29,0.53,10.1,6
2902,white,NaN,0.360,0.14,8.9,0.036,38.0,155.0,0.99622,3.27,NaN,9.4,5
6428,red,NaN,0.440,0.09,2.2,0.063,9.0,18.0,0.99444,NaN,0.69,11.3,6
6429,red,NaN,0.705,0.10,2.8,0.081,13.0,28.0,0.99631,NaN,0.66,10.2,5


In [8]:
print("white mean: {}".format(df[df['type'] == 'white']['fixed_acidity'].mean()))
print("red mean: {}".format(df[df['type'] == 'red']['fixed_acidity'].mean()))

white mean: 6.855531697341513
red mean: 8.32210394489668


In [9]:
df.loc[(df['fixed_acidity'].isna()) & (df['type'] == 'white'), 'fixed_acidity'] = df[df['type'] == 'white']['fixed_acidity'].mean()
df.loc[(df['fixed_acidity'].isna()) & (df['type'] == 'red'), 'fixed_acidity'] = df[df['type'] == 'red']['fixed_acidity'].mean()

In [10]:
# проверяем отсутствие пропусков fixed_acidity
df.isna().sum()

type                    0
fixed_acidity           0
volatile_acidity        8
citric_acid             3
residual_sugar          2
chlorides               2
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      9
sulphates               4
alcohol                 0
quality                 0
dtype: int64

Посмотрим на пропуски в столбце `volatile_acidity`, видим что пропуски содержатся по двум типам вина - красное и белое, а также можно заметить что значения в столбце `free_sulfur_dioxide` варьируется от 8 до 55. 

In [11]:
df[df['volatile_acidity'].isna()]

,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
86,white,7.200000,NaN,0.63,11.0,0.044,55.0,156.0,0.99740,3.09,0.44,8.7,6
521,white,7.900000,NaN,0.26,2.1,0.039,8.0,143.0,0.99420,3.05,0.74,9.8,5
621,white,6.500000,NaN,0.43,8.9,0.083,50.0,171.0,0.99650,2.85,0.50,9.0,5
812,white,6.400000,NaN,0.28,1.1,0.055,9.0,160.0,0.99405,3.42,0.50,9.1,7
1079,white,6.855532,NaN,0.29,6.2,0.046,29.0,227.0,0.99520,3.29,0.53,10.1,6
2894,white,6.400000,NaN,0.36,1.4,0.044,22.0,68.0,0.99014,3.15,0.50,11.7,7
4895,white,6.500000,NaN,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
6486,red,7.200000,NaN,0.33,2.5,0.068,34.0,102.0,0.99414,3.27,0.78,12.8,6


Можно разделить эти строки на 3 диапазона: от 5 до 15, от 20 до 40, и от 45 до 60, и посчитать по ним среднее.

In [12]:
print("white, sulfur  between 5-15 mean: {}".format(df.loc[(df['type'] == 'white') 
                                                           & (df['free_sulfur_dioxide'] > 5) 
                                                           & (df['free_sulfur_dioxide'] < 15), 
                                                           'volatile_acidity'].mean()))

white, sulfur  between 5-15 mean: 0.30465736040609137


In [13]:
print("white, sulfur  between 20-40 mean: {}".format(df.loc[(df['type'] == 'white') 
                                                            & (df['free_sulfur_dioxide'] > 20) 
                                                            & (df['free_sulfur_dioxide'] < 40), 
                                                            'volatile_acidity'].mean()))

white, sulfur  between 20-40 mean: 0.27927849264705884


In [14]:
print("white, sulfur  between 45-60 mean: {}".format(df.loc[(df['type'] == 'white') 
                                                            & (df['free_sulfur_dioxide'] > 45) 
                                                            & (df['free_sulfur_dioxide'] < 60), 
                                                            'volatile_acidity'].mean()))

white, sulfur  between 45-60 mean: 0.26686331775700933


In [15]:
print("red, sulfur  between 20-40 mean: {}".format(df.loc[(df['type'] == 'red') 
                                                          & (df['free_sulfur_dioxide'] > 20) 
                                                          & (df['free_sulfur_dioxide'] < 40), 
                                                          'volatile_acidity'].mean()))

red, sulfur  between 20-40 mean: 0.5209079283887468


In [16]:
df.loc[(df['volatile_acidity'].isna()) & (df['type'] == 'white') 
                                       & (df['free_sulfur_dioxide'] > 5) & (df['free_sulfur_dioxide'] < 15), 
                                       'volatile_acidity'] = df.loc[(df['type'] == 'white') 
                                       & (df['free_sulfur_dioxide'] > 5) & (df['free_sulfur_dioxide'] < 15), 
                                      'volatile_acidity'].mean()

In [17]:
df.loc[(df['volatile_acidity'].isna()) & (df['type'] == 'white') 
                                       & (df['free_sulfur_dioxide'] > 20) & (df['free_sulfur_dioxide'] < 40), 
                                       'volatile_acidity'] = df.loc[(df['type'] == 'white') 
                                       & (df['free_sulfur_dioxide'] > 20) & (df['free_sulfur_dioxide'] < 40), 
                                       'volatile_acidity'].mean()

In [18]:
df.loc[(df['volatile_acidity'].isna()) & (df['type'] == 'white') 
                                       & (df['free_sulfur_dioxide'] > 45) & (df['free_sulfur_dioxide'] < 60), 
                                       'volatile_acidity'] = df.loc[(df['type'] == 'white') 
                                       & (df['free_sulfur_dioxide'] > 45) & (df['free_sulfur_dioxide'] < 60), 
                                       'volatile_acidity'].mean()

In [19]:
df.loc[(df['volatile_acidity'].isna()) & (df['type'] == 'red') 
                                       & (df['free_sulfur_dioxide'] > 20) & (df['free_sulfur_dioxide'] < 40), 
                                       'volatile_acidity'] = df.loc[(df['type'] == 'red') 
                                       & (df['free_sulfur_dioxide'] > 20) & (df['free_sulfur_dioxide'] < 40), 
                                       'volatile_acidity'].mean() 

In [20]:
# проверяем отсутствие пропусков volatile_acidity
df.isna().sum()

type                    0
fixed_acidity           0
volatile_acidity        0
citric_acid             3
residual_sugar          2
chlorides               2
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      9
sulphates               4
alcohol                 0
quality                 0
dtype: int64

Посмотрим на пропуски в столбце `citric_acid`, видим что пропуски содержатся по двум типам вина - красное и белое, а также можно заметить что значения в столбце `residual_sugar` варьируется от 2 до 13. 

In [21]:
df[df['citric_acid'].isna()]

,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
268,white,5.3,0.585,NaN,7.1,0.044,34.0,145.0,0.9945,3.34,0.57,9.7,6
909,white,7.5,0.240,NaN,13.0,0.049,46.0,217.0,0.9985,3.08,0.53,8.8,5
6320,red,7.0,0.540,NaN,2.1,0.079,39.0,55.0,0.9956,3.39,0.84,11.4,6


Можно разделить эти строки на 3 диапазона: меньше 5, от 5 до 10, от 10 до 15, и посчитать по ним среднее.

In [22]:
print("white, sugar between 20-40 mean: {}".format(df.loc[(df['type'] == 'white') 
                                                          & (df['residual_sugar'] > 5) 
                                                          & (df['free_sulfur_dioxide'] <= 10), 
                                                          'citric_acid'].mean()))

white, sugar between 20-40 mean: 0.3408928571428572


In [23]:
print("white, sugar between 45-60 mean: {}".format(df.loc[(df['type'] == 'white') 
                                                          & (df['residual_sugar'] > 10) 
                                                          & (df['free_sulfur_dioxide'] < 15), 
                                                          'citric_acid'].mean()))

white, sugar between 45-60 mean: 0.36749999999999994


In [24]:
print("red, sugar between 20-40 mean: {}".format(df.loc[(df['type'] == 'red') 
                                                        & (df['free_sulfur_dioxide'] <= 5), 
                                                        'citric_acid'].mean()))

red, sugar between 20-40 mean: 0.3003030303030303


In [25]:
df.loc[(df['citric_acid'].isna()) & (df['type'] == 'white') 
                                  & (df['residual_sugar'] > 5) & (df['residual_sugar'] <= 10), 
                                  'citric_acid'] = df.loc[(df['type'] == 'white') & (df['residual_sugar'] > 5) 
                                  & (df['free_sulfur_dioxide'] <= 10), 'citric_acid'].mean()

In [26]:
df.loc[(df['citric_acid'].isna()) & (df['type'] == 'white') 
                                  & (df['residual_sugar'] > 10) & (df['residual_sugar'] < 15), 
                                  'citric_acid'] = df.loc[(df['type'] == 'white') & (df['residual_sugar'] > 10) 
                                  & (df['free_sulfur_dioxide'] < 15), 'citric_acid'].mean()

In [27]:
df.loc[(df['citric_acid'].isna()) & (df['type'] == 'red') 
                                  & (df['residual_sugar'] <= 5), 'citric_acid'] = df.loc[(df['type'] == 'red') 
                                  & (df['free_sulfur_dioxide'] <= 5), 'citric_acid'].mean()


In [28]:
# проверяем отсутствие пропусков citric_acid
df.isna().sum()

type                    0
fixed_acidity           0
volatile_acidity        0
citric_acid             0
residual_sugar          2
chlorides               2
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      9
sulphates               4
alcohol                 0
quality                 0
dtype: int64

Посмотрим на пропуски в столбце `residual_sugar`, видим что пропуски содержатся по типу вина - `white`. 

In [29]:
df[df['residual_sugar'].isna()]

,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
33,white,6.2,0.120,0.34,NaN,0.045,43.0,117.0,0.9939,3.42,0.51,9.0,6
438,white,7.4,0.155,0.34,NaN,0.045,73.5,214.0,0.9934,3.18,0.61,9.9,7


Заменим пустые значения в столбце `residual_sugar` на среднее значение по типу вина - `white`.

In [30]:
print("white: {}".format(df.loc[df['type'] == 'white', 'residual_sugar'].mean()))

white: 6.3932495915032685


In [32]:
df.loc[(df['residual_sugar'].isna()) & (df['type'] == 'white'), 
                                     'residual_sugar'] = df.loc[df['type'] == 'white', 'residual_sugar'].mean()

In [33]:
# проверяем отсутствие пропусков residual_sugar
df.isna().sum()

type                    0
fixed_acidity           0
volatile_acidity        0
citric_acid             0
residual_sugar          0
chlorides               2
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      9
sulphates               4
alcohol                 0
quality                 0
dtype: int64

Посмотрим на пропуски в столбце `chlorides`. 

In [41]:
df[df['chlorides'].isna()]

,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality


Видим что пропуски содержатся по типу вина - white, а также можно заметить что значения в столбце `free sulfur dioxide` - 4, 56. Можно посчитать среднее по столбцу `free sulfur dioxide` для этих строк в 2 диапазонах: меньше 10 и от 50 до 60.

In [42]:
print("white, free_sulfur_dioxide < 10: {}".format(df.loc[(df['type'] == 'white') 
                                                          & (df['free_sulfur_dioxide'] < 10), 'chlorides'].mean()))

white, free_sulfur_dioxide < 10: 0.042976047904191614


In [43]:
print("white, free_sulfur_dioxide between 50-60: {}".format(df.loc[(df['type'] == 'white') 
                                                                   & (df['free_sulfur_dioxide'] > 50) & (df['free_sulfur_dioxide'] < 60), 
                                                                   'chlorides'].mean()))

white, free_sulfur_dioxide between 50-60: 0.047722567287784685


In [44]:
df.loc[(df['chlorides'].isna()) & (df['type'] == 'white') & (df['free_sulfur_dioxide'] < 10), 
                                                          'chlorides'] = df.loc[(df['type'] == 'white') 
                                                          & (df['free_sulfur_dioxide'] < 10), 'chlorides'].mean()

In [45]:
df.loc[(df['chlorides'].isna()) & (df['type'] == 'white') & (df['free_sulfur_dioxide'] > 50) 
                                                          & (df['free_sulfur_dioxide'] < 60), 
                                                          'chlorides'] = df.loc[(df['type'] == 'white') 
                                                          & (df['free_sulfur_dioxide'] > 50) 
                                                          & (df['free_sulfur_dioxide'] < 60), 'chlorides'].mean()

In [46]:
# проверяем отсутствие пропусков chlorides
df.isna().sum()

type                    0
fixed_acidity           0
volatile_acidity        0
citric_acid             0
residual_sugar          0
chlorides               0
free_sulfur_dioxide     0
total_sulfur_dioxide    0
density                 0
pH                      9
sulphates               4
alcohol                 0
quality                 0
dtype: int64